In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import metrics
import warnings
from q1_old import KNNClassifier as knc

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [9]:
class SVMclassifier:
    featuresd_scale = []
    labelsd = []
    # scaler = MinMaxScaler()
    scaler = StandardScaler()
    pca = PCA(n_components=500)
    knn_classifier = knc()


    def stat(self, labelst, labelsp):
        print("Accuracy:",metrics.accuracy_score(labelst, labelsp))
        print("F1 Score",metrics.f1_score(labelst, labelsp, average='micro'))
        print("Confusion matrix")
        print(metrics.confusion_matrix(labelst, labelsp))
        print("-----------------------------------------------------")

    def svmC(self, c):
        subclf = SVC(kernel='linear', C=c)
        subclf.fit(self.featuresd_scale,self.labelsd)
        return subclf
    
    def preprocess(self, file):
        with open(file, 'rb') as fo:
            batch = pickle.load(fo, encoding='latin1')
            features = batch['data']
            labels = batch['labels']
            return features, labels

    def train(self):
        features1, labels1 = self.preprocess("./cifar-10-batches-py/data_batch_1")
        features2, labels2 = self.preprocess("./cifar-10-batches-py/data_batch_2")
        features3, labels3 = self.preprocess("./cifar-10-batches-py/data_batch_3")
        features4, labels4 = self.preprocess("./cifar-10-batches-py/data_batch_4")
        features5, labels5 = self.preprocess("./cifar-10-batches-py/data_batch_5")
        featuresd = np.concatenate((features1, features2, features3, features4, features5))
        labelsd = np.concatenate((labels1, labels2, labels3, labels4, labels5))
        featuresd = featuresd[0:5000]
        labelsd = labelsd[0:5000]
#         print("features shape",featuresd.shape)
#         print("labels shape", labelsd.shape)
        featuresd_pca = self.pca.fit_transform(featuresd)
        self.featuresd_scale = self.scaler.fit_transform(featuresd_pca)
        self.labelsd = np.array(labelsd)
#         print("features scale shape",self.featuresd_scale.shape)
#         print("training done")

    def test(self):
        featurest, labelst = self.preprocess("./cifar-10-batches-py/test_batch")
        featurest = featurest[0:1000]
        labelst = labelst[0:1000]
        featurest_pca = self.pca.transform(featurest)
        featurest_scale = self.scaler.transform(featurest_pca)
#         print("featurest scale shape", featurest_scale.shape)
        labelst = np.array(labelst)

        subclf = self.svmC(0.1)
        labelsp = subclf.predict(featurest_scale)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.1")
        self.stat(labelst, labelsp)

        subclf = self.svmC(0.01)
        labelsp = subclf.predict(featurest_scale)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.01")
        self.stat(labelst, labelsp)
    
        subclf = self.svmC(0.001)
        l = subclf.support_vectors_
        print("Length of support vector images ",len(l))
        print("-----------------------------------------------------")
        print("Support vectors are ",l)
        labelsp = subclf.predict(featurest_scale)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.001")
        self.stat(labelst, labelsp)

        labelspKNN = self.knn_classifier.test(self.featuresd_scale, self.labelsd, featurest_scale)
        print("-----------------------------------------------------")
        print("Comparison with KNN")
        self.stat(labelst, labelspKNN)


In [10]:
svm_classifier = SVMclassifier()
svm_classifier.train()
svm_classifier.test()

-----------------------------------------------------
SVM : For c value 0.1
Accuracy: 0.266
F1 Score 0.266
Confusion matrix
[[28  1  9  7  6  8  7  7 20 10]
 [11 30  4 10  4  5  3  7  6  9]
 [13  6 30 13 14  5  5  7  5  2]
 [ 4 12 12 19 10 20 11  7  5  3]
 [11  3 18  9 19  5 11 10  2  2]
 [ 7  7 10 22 15 11  4  6  3  1]
 [ 9  7  7 17 17 13 30  6  3  3]
 [14  3 10  8 21 10  4 23  3  6]
 [16 11  4  4  4  3  2  5 50  7]
 [ 8 25  7 12  3  4  3  7 14 26]]
-----------------------------------------------------
-----------------------------------------------------
SVM : For c value 0.01
Accuracy: 0.291
F1 Score 0.291
Confusion matrix
[[37  1 13  9  6  6  3  7 12  9]
 [ 5 35  4  8  3  7  3  7  9  8]
 [12  6 31 13 14  8  6  2  7  1]
 [ 3  8 12 20 10 23 11  6  5  5]
 [ 7  2 21 11 17  7 10 10  2  3]
 [ 6  4 14 18 14 10  8  6  5  1]
 [ 5  6 16 20 13 11 31  6  2  2]
 [ 8  2  8 10 16 12  8 28  5  5]
 [21  7  2  4  3  6  2  3 49  9]
 [ 7 25  5  7  6  4  2  5 15 33]]
-----------------------------------

As clearly seen in the above SVM classifier(based on sklearn), as the c value decreases, the accuracy score increases. The best condition is for c=0.001. It is better than my KNN classifier. The linear SVM is made better by preprocessing on the data. This preprocessing includes standardization and pca implementation to have the best 500 features. Instead of standardization, MinMax scaler can also be used. Support vector images are also listed out for c = 0.001.

SVM without the preprocessing.

In [15]:
class SVMclassifier_without:
    featuresd = []
    labelsd = []
    knn_classifier = knc()


    def stat(self, labelst, labelsp):
        print("Accuracy:",metrics.accuracy_score(labelst, labelsp))
        print("F1 Score",metrics.f1_score(labelst, labelsp, average='micro'))
        print("Confusion matrix")
        print(metrics.confusion_matrix(labelst, labelsp))
        print("-----------------------------------------------------")

    def svmC(self, c):
        subclf = SVC(kernel='linear', C=c)
        subclf.fit(self.featuresd,self.labelsd)
        return subclf
    
    def preprocess(self, file):
        with open(file, 'rb') as fo:
            batch = pickle.load(fo, encoding='latin1')
            features = batch['data']
            labels = batch['labels']
            return features, labels

    def train(self):
        features1, labels1 = self.preprocess("./cifar-10-batches-py/data_batch_1")
        features2, labels2 = self.preprocess("./cifar-10-batches-py/data_batch_2")
        features3, labels3 = self.preprocess("./cifar-10-batches-py/data_batch_3")
        features4, labels4 = self.preprocess("./cifar-10-batches-py/data_batch_4")
        features5, labels5 = self.preprocess("./cifar-10-batches-py/data_batch_5")
        featuresd = np.concatenate((features1, features2, features3, features4, features5))
        labelsd = np.concatenate((labels1, labels2, labels3, labels4, labels5))
        self.featuresd = featuresd[0:5000]
        self.labelsd = labelsd[0:5000]

    def test(self):
        featurest, labelst = self.preprocess("./cifar-10-batches-py/test_batch")
        featurest = featurest[0:1000]
        labelst = labelst[0:1000]
        labelst = np.array(labelst)

        subclf = self.svmC(0.1)
        labelsp = subclf.predict(featurest)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.1")
        self.stat(labelst, labelsp)

        subclf = self.svmC(0.01)
        labelsp = subclf.predict(featurest)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.01")
        self.stat(labelst, labelsp)
    
        subclf = self.svmC(0.001)
        l = subclf.support_vectors_
        print("Length of support vector images ",len(l))
        print("-----------------------------------------------------")
        print("Support vectors are ",l)
        labelsp = subclf.predict(featurest)
        print("-----------------------------------------------------")
        print("SVM : For c value 0.001")
        self.stat(labelst, labelsp)

        labelspKNN = self.knn_classifier.test(self.featuresd, self.labelsd, featurest)
        print("-----------------------------------------------------")
        print("Comparison with KNN")
        self.stat(labelst, labelspKNN)


In [16]:
svm_classifier_without = SVMclassifier_without()
svm_classifier_without.train()
svm_classifier_without.test()

-----------------------------------------------------
SVM : For c value 0.1
Accuracy: 0.299
F1 Score 0.299
Confusion matrix
[[38  3  9  4  4  4  7  7 21  6]
 [ 5 38  3  7  2  6  2  6  6 14]
 [14  2 31 11 16  9  9  4  4  0]
 [ 3  8  8 20 14 20 11  8  6  5]
 [11  5 18 11 25  4  5  7  4  0]
 [10  4 10 23 12  9  7  7  2  2]
 [ 5  6 13 19 17 12 33  2  3  2]
 [13  4 13 10 13  8  7 30  2  2]
 [22  5  5  2  5  7  2  3 47  8]
 [ 8 28  3  5  4  3  1  9 20 28]]
-----------------------------------------------------
-----------------------------------------------------
SVM : For c value 0.01
Accuracy: 0.299
F1 Score 0.299
Confusion matrix
[[38  3  9  4  4  4  7  7 21  6]
 [ 5 38  3  7  2  6  2  6  6 14]
 [14  2 31 11 16  9  9  4  4  0]
 [ 3  8  8 20 14 20 11  8  6  5]
 [11  5 18 11 25  4  5  7  4  0]
 [10  4 10 23 12  9  7  7  2  2]
 [ 5  6 13 19 17 12 33  2  3  2]
 [13  4 13 10 13  8  7 30  2  2]
 [22  5  5  2  5  7  2  3 47  8]
 [ 8 28  3  5  4  3  1  9 20 28]]
-----------------------------------